# **Default Setting**

* Python: 3.6.9
* ML Framework: tf-nightly-gpu 2.5.0-dev20201208
* CPU: AMD Ryzen 5 5600X 6-Core Processor
* GPU: GeForce RTX 3070 (8G) (CUDA 11.1)
* RAM: 32G
* Platform: linux 20.04 LTS

In [1]:
%env NOTEBOOKNAME try8
%env LOCAL_DATA_PATH data
%env SUBMISSION_PATH submission

env: NOTEBOOKNAME=try8
env: LOCAL_DATA_PATH=data
env: SUBMISSION_PATH=submission


In [2]:
import tensorflow as tf
import tensorflow_addons as tfa

import datetime
import glob
import os
import platform

import numpy as np
import pandas as pd

from collections import OrderedDict
from sklearn.model_selection import train_test_split

print(f"tf.__version__: {tf.__version__}")
print(f"tfa.__version__: {tfa.__version__}")

tf.__version__: 2.5.0-dev20201208
tfa.__version__: 0.11.2


/home/myunggyo/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow_addons/utils/ensure_tf_install.py:44: UserWarning: You are currently using a nightly version of TensorFlow (2.5.0-dev20201208). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  UserWarning,


In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# !python -V

In [5]:
# !cat /proc/cpuinfo

In [6]:
!nvidia-smi

Sat Dec 12 16:44:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 455.32.00    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Graphics Device     On   | 00000000:0A:00.0  On |                  N/A |
|  0%   37C    P8    24W / 220W |    248MiB /  7979MiB |     11%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
# !df -h

In [8]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           31Gi       3.4Gi        24Gi       409Mi       3.4Gi        27Gi
Swap:         2.0Gi          0B       2.0Gi


In [9]:
# platform.platform()

In [10]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

# **Define Arguments**

In [11]:
args = OrderedDict({
    # Basic
    "SEED": 42,
    "TEST_SIZE": 0.2,
    "EPOCH": 10,
    "INIT_LR": 1e-3,
    
    # Dataframe
    "NUM_TEST_CSV": 81,
    
    # Dataset Pipeline
    "WINDOW_INP": 7 * 24 * 2, # 336, 7 days
    "WINDOW_TAR": 2 * 24 * 2, # 96, 2 days
    "WINDOW_SHIFT": 1 * 24 * 2, # 1 day
    "WINDOW_STRIDE": 1,
    "WINDOW_DR": True, # Drop remainders
    "AUTO": tf.data.experimental.AUTOTUNE,
    
    "GLOBAL_BATCH_SIZE": 128,
    
    # Model
    "NUM_UNITS": 128, # lstm units
    "NUM_FEATURES": 6, # dense featues
    "NUM_QUANTILE": 9,
    
})

args

OrderedDict([('SEED', 42),
             ('TEST_SIZE', 0.2),
             ('EPOCH', 10),
             ('INIT_LR', 0.001),
             ('NUM_TEST_CSV', 81),
             ('WINDOW_INP', 336),
             ('WINDOW_TAR', 96),
             ('WINDOW_SHIFT', 48),
             ('WINDOW_STRIDE', 1),
             ('WINDOW_DR', True),
             ('AUTO', -1),
             ('GLOBAL_BATCH_SIZE', 128),
             ('NUM_UNITS', 128),
             ('NUM_FEATURES', 6),
             ('NUM_QUANTILE', 9)])

# **Load Datasets**

## **Train / Validation**

In [12]:
df = pd.read_csv(os.path.join(os.environ["LOCAL_DATA_PATH"], "train", "train.csv"))
df.head()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.0
1,0,0,30,0,0,1.5,69.06,-12,0.0
2,0,1,0,0,0,1.6,71.78,-12,0.0
3,0,1,30,0,0,1.6,71.75,-12,0.0
4,0,2,0,0,0,1.6,75.20,-12,0.0


In [13]:
df.describe()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
count,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000
mean,547.000000,11.500000,15.000000,64.344121,234.792371,2.456033,56.793102,9.279928,17.790630
std,316.102148,6.922252,15.000143,103.897125,349.684583,1.426874,22.052875,10.179741,25.759955
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.590000,-19.000000,0.000000
25%,273.000000,5.750000,0.000000,0.000000,0.000000,1.400000,39.697500,1.000000,0.000000
50%,547.000000,11.500000,15.000000,0.000000,0.000000,2.200000,57.600000,9.000000,0.000000
75%,821.000000,17.250000,30.000000,87.000000,469.000000,3.200000,72.770000,17.000000,32.089890
max,1094.000000,23.000000,30.000000,528.000000,1059.000000,12.000000,100.000000,35.000000,99.913939


In [24]:
tr_df, vl_df = train_test_split(
    df, 
    test_size = args["TEST_SIZE"], 
    random_state = args["SEED"])

tr_df.shape, vl_df.shape

((42048, 9), (10512, 9))

In [25]:
def preprocessing(df, is_training = True):
    # Drop timestampes.
    try:
        df = df.drop(["Day", "Hour", "Minute"], axis = 1)
    except:
        pass
    
    # Standarize.
    for column in df.columns[:-1]:
        if is_training:
            args[f"{column}_MEAN"] = df[column].mean()
            args[f"{column}_STD"] = df[column].std()
        
        df[column] = (df[column] - args[f"{column}_MEAN"]) / args[f"{column}_STD"]
        
    return df

In [26]:
tr_df = preprocessing(tr_df)
vl_df = preprocessing(vl_df, is_training = False)

In [27]:
tr_df.describe()

,DHI,DNI,WS,RH,T,TARGET
count,4.204800e+04,4.204800e+04,4.204800e+04,4.204800e+04,4.204800e+04,42048.000000
mean,-1.064597e-17,-1.199784e-17,1.203164e-16,2.484061e-16,-5.660955e-17,17.719547
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,25.678521
min,-6.191629e-01,-6.709268e-01,-1.723166e+00,-2.229873e+00,-2.775532e+00,0.000000
25%,-6.191629e-01,-6.709268e-01,-7.399790e-01,-7.769368e-01,-8.107883e-01,0.000000
50%,-6.191629e-01,-6.709268e-01,-1.781579e-01,3.674375e-02,-2.489105e-02,0.000000
75%,2.229109e-01,6.726452e-01,5.241185e-01,7.271943e-01,7.610062e-01,31.997818
max,4.491354e+00,2.356398e+00,6.704151e+00,1.956776e+00,2.529275e+00,99.747138


In [28]:
vl_df.describe()

,DHI,DNI,WS,RH,T,TARGET
count,10512.000000,10512.000000,10512.000000,10512.000000,10512.000000,10512.000000
mean,0.018122,0.001328,0.008241,-0.003595,0.013041,18.074962
std,1.027720,0.998199,1.010278,0.995579,1.000124,26.082442
min,-0.619163,-0.670927,-1.652938,-2.211298,-2.579057,0.000000
25%,-0.619163,-0.670927,-0.739979,-0.771953,-0.810788,0.000000
50%,-0.619163,-0.670927,-0.178158,0.033572,-0.024891,0.000000
75%,0.242269,0.655493,0.524119,0.701823,0.761006,32.370490
max,4.442959,2.350681,6.563695,1.956776,2.529275,99.913939


## **Test**

In [29]:
ts_filenames = [os.path.join(os.environ["LOCAL_DATA_PATH"], "test", f"{i}.csv") for i in range(args["NUM_TEST_CSV"])]
ts_dfs = [preprocessing(pd.read_csv(ts_filename), is_training = False) for ts_filename in ts_filenames]
ts_df = pd.concat(ts_dfs)

In [30]:
ts_df.head()

,DHI,DNI,WS,RH,T,TARGET
0,-0.619163,-0.670927,0.172980,-1.014336,-0.909025,0.0
1,-0.619163,-0.670927,0.172980,-1.025662,-0.899202,0.0
2,-0.619163,-0.670927,0.172980,-1.022944,-0.889378,0.0
3,-0.619163,-0.670927,0.172980,-1.033817,-0.879554,0.0
4,-0.619163,-0.670927,0.243208,-1.034723,-0.869731,0.0


In [31]:
ts_df.describe()

,DHI,DNI,WS,RH,T,TARGET
count,27216.000000,27216.000000,27216.000000,27216.000000,27216.000000,27216.000000
mean,-0.047397,0.046952,-0.127593,-0.471834,0.048424,18.235096
std,0.886646,1.023778,0.893017,0.868807,1.108719,26.115609
min,-0.619163,-0.670927,-1.652938,-2.351291,-2.451349,0.000000
25%,-0.619163,-0.670927,-0.810207,-1.165768,-0.869731,0.000000
50%,-0.561089,-0.670927,-0.318613,-0.452099,-0.054362,0.563069
75%,0.184195,0.775557,0.313436,0.184439,0.888715,32.557286
max,4.288095,2.413571,6.001874,1.620159,2.696278,99.450872


# **Make Dataset Pipelines**

In [32]:
@tf.function
def _split_window(features):
    return tf.split(features, [5, 1], axis = 0)

In [33]:
tr_tensor = tf.constant(tr_df, dtype = tf.float32)
vl_tensor = tf.constant(vl_df, dtype = tf.float32)
ts_tensor = tf.constant(ts_df, dtype = tf.float32)

tr_dataset = tf.data.Dataset.from_tensor_slices(tr_tensor
#                     ).window(args["WINDOW_INP"] + args["WINDOW_TAR"], args["WINDOW_SHIFT"], args["WINDOW_STRIDE"], args["WINDOW_DR"]).flat_map(_flat_fn_tr
                    ).map(_split_window, num_parallel_calls = args["AUTO"]
                    ).batch(args["GLOBAL_BATCH_SIZE"]
                    ).cache(
                    ).prefetch(args["AUTO"])

vl_dataset = tf.data.Dataset.from_tensor_slices(vl_tensor
#                     ).window(args["WINDOW_INP"] + args["WINDOW_TAR"], args["WINDOW_SHIFT"], args["WINDOW_STRIDE"], args["WINDOW_DR"]).flat_map(_flat_fn_tr
                    ).map(_split_window, num_parallel_calls = args["AUTO"]
                    ).batch(args["GLOBAL_BATCH_SIZE"]
                    ).cache(
                    ).prefetch(args["AUTO"])

ts_dataset = tf.data.Dataset.from_tensor_slices(ts_tensor
#                     ).window(args["WINDOW_INP"], args["WINDOW_INP"], args["WINDOW_STRIDE"], args["WINDOW_DR"]).flat_map(_flat_fn_ts
                    ).map(_split_window, num_parallel_calls = args["AUTO"]
                    ).batch(args["GLOBAL_BATCH_SIZE"]
#                     ).cache(
                    ).prefetch(args["AUTO"])

# Print the shapes
print(f"tr_dataset: {tr_dataset.element_spec}")
print(f"vl_dataset: {vl_dataset.element_spec}")
print(f"ts_dataset: {ts_dataset.element_spec}")

tr_dataset: (TensorSpec(shape=(None, 5), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))
vl_dataset: (TensorSpec(shape=(None, 5), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))
ts_dataset: (TensorSpec(shape=(None, 5), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))


## **Take Samples**

In [41]:
for element in tr_dataset.take(1):
    foo, bar = element
    print(foo.shape, bar.shape)

(128, 5) (128, 1)


In [42]:
for element in vl_dataset.take(1):
    foo, bar = element
    print(foo.shape, bar.shape)

(128, 5) (128, 1)


In [43]:
for element in ts_dataset.take(1):
    foo, bar = element
    print(foo.shape, bar.shape)

(128, 5) (128, 1)


# **Create Network**

In [34]:
class TARGET_MODEL(tf.keras.Model):
    def __init__(self, units, name):
        super(TARGET_MODEL, self).__init__(name = name)
        self.units = units
        
        self.dense1 = tf.keras.layers.Dense(units = self.units, activation = "relu")
        self.dense2 = tf.keras.layers.Dense(units = self.units, activation = "relu")
        self.dense3 = tf.keras.layers.Dense(units = 1)
        
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        
        return x        

# **Compile and Fit**

   * Repeat the last 24 hours.

In [35]:
model = TARGET_MODEL(
    units = 64,
    name = "model")

model.compile(
    loss = tf.keras.losses.MAE,
    optimizer = tf.keras.optimizers.Adam())

In [36]:
# TensorBoard callback.
log_dir = os.path.join(
    "logs", "fit", os.environ["NOTEBOOKNAME"], 
    datetime.datetime.now().strftime(f"{model.name}-%Y%m%d-%H%M%S"))
tb_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1)

_ = model.fit(
    tr_dataset,
    validation_data = vl_dataset,
    epochs = 500,
    verbose = 0,
    callbacks = [tb_callback])

In [37]:
%tensorboard --logdir logs/fit --host jupyter-server

Reusing TensorBoard on port 6006 (pid 3556), started -1 day, 16:18:40 ago. (Use '!kill 3556' to kill it.)

# **Predict**

In [38]:
model.evaluate(ts_dataset, verbose = 1)

213/213 [==============================] - 0s 797us/step - loss: 0.6481


0.6481271386146545

In [57]:
preds = [np.array(model.predict(ts_dataset))[..., -1] for model in models]
preds[0].shape

(81, 96)

In [58]:
submission = pd.read_csv(os.path.join(os.environ["LOCAL_DATA_PATH"], "sample_submission.csv"), index_col = "id")
submission.head()

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
id,,,,,,,,,
0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
for column, pred in zip(submission.columns, preds):
    foo = np.reshape(pred, (-1,))
    foo = np.where(foo < 0, 0, foo) # clip by value
    
    submission[column] = foo

In [66]:
submission.head()

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
id,,,,,,,,,
0.csv_Day7_0h00m,0.019628,0.009365,0.0,0.000000,0.112591,7.637287,14.906515,17.131758,17.608368
0.csv_Day7_0h30m,0.008961,0.009944,0.0,0.000000,0.124864,7.905676,15.397655,17.560627,17.959702
0.csv_Day7_1h00m,0.001013,0.008628,0.0,0.004815,0.133084,8.147173,15.504680,17.658821,18.036112
0.csv_Day7_1h30m,0.000000,0.007088,0.0,0.010220,0.139264,8.294870,15.531036,17.690212,18.056557
0.csv_Day7_2h00m,0.000000,0.005681,0.0,0.013636,0.143860,8.387443,15.534620,17.701130,18.060827


In [67]:
submission.to_csv(
    os.path.join(os.environ["SUBMISSION_PATH"], f"{os.environ['NOTEBOOKNAME']}_submission.csv"))